## Connect with the Python Driver
- Install the DataStax Python Driver

- Verify that the DataStax Python driver installed successfully:

In [1]:
import cassandra
print(cassandra.__version__)

3.29.0


In [4]:
import os
import asyncio
from cassandra.cluster import Cluster
from dotenv import load_dotenv
from cassandra.auth import PlainTextAuthProvider
import json

# Custom event loop configuration (for Python 3.12 or newer)
class CustomEventLoop(asyncio.AbstractEventLoop):
    def __init__(self):
        super().__init__()

    def create_task(self, *args, **kwargs):
        return asyncio.create_task(*args, **kwargs, loop=self)

loop = CustomEventLoop()
cassandra.io.SelectorManager._event_loop = loop

# Load environment variables
load_dotenv()

# Retrieve credentials
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
SECURE_CONNECT_BUNDLE = os.getenv("SECURE_CONNECT_BUNDLE")

# Check for required credentials
if not all([CLIENT_ID, CLIENT_SECRET, SECURE_CONNECT_BUNDLE]):
    raise ValueError("Missing required credentials in .env file!")

# Cluster configuration
cloud_config = {'secure_connect_bundle': SECURE_CONNECT_BUNDLE}

# Authentication provider
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)

async def main():
    """Asynchronous function using trio"""
    try:
        # Use connect() with custom loop for Python 3.12 or newer
        cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, loop=loop)
        async with cluster.connect() as session:
            # Execute your queries here
            row = await session.execute("select release_version from system.local").one()
            if row:
                print(row[0])
            else:
                print("An error occurred.")
    except Exception as e:
        print(f"Error connecting: {e}")

if __name__ == "__main__":
    import trio
    trio.run(main)

DependencyException: Unable to load a default connection class
The following exceptions were observed: 
 - The C extension needed to use libev was not found.  This probably means that you didn't have the required build dependencies when installing the driver.  See http://datastax.github.io/python-driver/installation.html#c-extensions for instructions on installing build dependencies and building the C extension.
 - Unable to import asyncore module.  Note that this module has been removed in Python 3.12 so when using the driver with this version (or anything newer) you will need to use one of the other event loop implementations.